In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=50,
  memory_key="chat_history"
)

In [5]:
template = """
  You are a helpful AI talking to a human.

  {chat_history}
  Human: {question}
  You:
"""

chain = LLMChain(
  llm=llm,
  memory=memory,
  prompt=PromptTemplate.from_template(template),
  verbose=True
)

In [6]:
chain.predict(question="내 이름은 홍길동이야")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful AI talking to a human.

  
  Human: 내 이름은 홍길동이야
  You:

안녕하세요, 홍길동님. 무엇을 도와드릴까요?
> Finished chain.
The human introduces themselves as 홍길동.

'안녕하세요, 홍길동님. 무엇을 도와드릴까요?'

In [7]:
chain.predict(question="나는 서자라서 아버지를 아버지라 부르지 못한단다.")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful AI talking to a human.

  System: The human introduces themselves as 홍길동.
AI: 안녕하세요, 홍길동님. 무엇을 도와드릴까요?
  Human: 나는 서자라서 아버지를 아버지라 부르지 못한단다.
  You:

그것은 안타깝네요. 가족 관계는 복잡할 수 있죠. 어떻게 도와드릴까요?
> Finished chain.
The human introduces themselves as 홍길동. The AI greets them and asks how it can help. 홍길동 explains that they cannot call their father "father" because they are a servant. The AI expresses sympathy for the complicated family situation and asks how it can assist.

'그것은 안타깝네요. 가족 관계는 복잡할 수 있죠. 어떻게 도와드릴까요?'

In [8]:
chain.predict(question="내 이름이 뭐라 그랬지?")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful AI talking to a human.

  System: The human introduces themselves as 홍길동. The AI greets them and asks how it can help. 홍길동 explains that they cannot call their father "father" because they are a servant. The AI expresses sympathy for the complicated family situation and asks how it can assist.
  Human: 내 이름이 뭐라 그랬지?
  You:

안녕하세요, 홍길동님. 어떻게 도와드릴까요?
> Finished chain.
The human introduces themselves as 홍길동. The AI greets them and asks how it can help. 홍길동 explains that they cannot call their father "father" because they are a servant. The AI expresses sympathy for the complicated family situation and asks how it can assist. 홍길동 then asks, "What did you say my name was?"

'안녕하세요, 홍길동님. 어떻게 도와드릴까요?'